## Questing Answering on SQuAD dataset

### Import libraries

In [26]:
import json
import pandas as pd
import os
import random

## Dataprocessing

#### Load dataset

In [21]:
def load_json(filename="training_set.json", folder="SQUAD MATERIAL"):    
    dataset_folder = os.path.join(os.getcwd(), folder)
    dataset_path = os.path.join(dataset_folder, filename)
    with open(dataset_path) as f:
        raw_json = json.load(f)

    return raw_json['data']

In [36]:
data = load_json()

#### Split dataset into train, val and test sets.
Splitting on title, so that all answers and questions in one title are in the same dataset

In [47]:
def split_dataset(data):
    random.shuffle(data)
    length_of_dataset = len(data)
    train_split = round(0.8*length_of_dataset)
    val_split = train_split + round(0.1*length_of_dataset)
    train_data = data[:train_split]
    val_data = data[train_split:val_split]
    test_data = data[val_split:]
    return train_data, val_data, test_data

In [48]:
train_data, val_data, test_data = split_dataset(data)

#### Create dataframe

In [54]:
def create_dataframe(data):
    contexts = []
    questions = []
    answers_text = []
    answers_start = []
    question_ids = []
    for i in range(len(data)):
        paragraphs = data[i]['paragraphs']
        for sub_para in paragraphs:
            for q_a in sub_para['qas']:
                questions.append(q_a['question'])
                q_a_answer_starts = []
                q_a_answers = []
                if len(q_a['answers'])>1:
                        print(q_a['answers'])
                for answer in q_a['answers']:
                    q_a_answer_starts.append(answer['answer_start'])
                    q_a_answers.append(answer['text'])
                answers_start.append(q_a_answer_starts)
                answers_text.append(q_a_answers)
                question_ids.append(q_a['id'])
                contexts.append(sub_para['context'])   
    df = pd.DataFrame({"questionID":question_ids, "context":contexts, "question": questions, "answer_start": answers_start, "text": answers_text})
    return df

In [55]:
df_train = create_dataframe(train_data)
df_val = create_dataframe(val_data)
df_test = create_dataframe(test_data)

#### Clean text

#### Make tokenixer